# Introduction

In order to successful knowledge discovery in databases (KDD), well-defined and formal methods should be applied for managing data.  Cross-industry standard process for data mining (CRISP-DM) model is a standard methodology, which includes six phases:
    1. Problem domain understanding
    2. Data Understanding
    3. Data Preparation
    4. Modeling
    5. Evaluation
    6. Deployment

----------------------------------------------------------------------------------------------------------------------

# Part 1 - Problem domain understanding

Cancer still remains a challenge for our world in preventing and treating. However, most of cancers are highly curable if they are detected early, so the stage at diagnosis heavily influences survival. Due to no early warning signs, it’s important to have routine screening tests. For many types of cancers such as colorectal cancer, lung cancer, stomach cancer, screening rate remains low due to unpleasant procedure and expensive cost. Therefore, a risk prediction model for cancer could bring benefits for both customer and health institute. For customer, it encourages people to take screening tests to detect the risk of cancer early and increase survival rate. For health institute, it provides more services and hence increase sale.

Nowadays, electronic medical records have become increasingly available through regular health checkup. In recent research, there has been an increasing interest in finding biomarkers of cancer from routine blood tests. In general, blood indices are related to cancer to some extent, but none of them solely exhibits a clear connection and can be used for diagnostic purposes. However, taking these basic blood indices together, information to be gleaned may reveal about converging signs or pattern of an individual for many forms of cancer. By monitoring selected biomarkers routinely measured in primary care, we can learn a lot about physiological patterns that promote carcinogenesis, proliferation, progression before tumor makers emerge.

This research aims to utilize temporal, longitudinal data accumulated in regular health checkup to explore pattern of change of many biomarkers in common blood test to predict cancer.

----------------------------------------------------------------------------------------------------------------------

# Part 2 - Data Exploration & Understanding

http://localhost:8888/notebooks/0-MyCollection/00-Sample/01-DataExploration.ipynb

## 1. Import Library and Define Common function

### 1.1. Import Library

In [295]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import pylab
from scipy import stats
from sklearn.linear_model import LinearRegression
import seaborn as sns
sns.set()

# Modelling Helpers:
# from sklearn.preprocessing import Imputer, Normalizer, scale
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score, ShuffleSplit, cross_validate
from sklearn import model_selection
from sklearn.model_selection import train_test_split

# Classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from lightgbm import LGBMClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
# Evaluation metrics for Classification
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, classification_report, roc_auc_score, roc_curve, precision_recall_curve, average_precision_score
from sklearn.metrics import mutual_info_score

# Regression
from sklearn.linear_model import LinearRegression,Ridge,Lasso,RidgeCV,ElasticNet,LogisticRegression
from sklearn.ensemble import RandomForestRegressor,BaggingRegressor,GradientBoostingRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
# Evaluation metrics for Regression 
from sklearn.metrics import mean_squared_log_error, mean_squared_error, r2_score, mean_absolute_error
from sklearn.metrics import (confusion_matrix, classification_report, accuracy_score, roc_auc_score, auc,
                             precision_score, recall_score, roc_curve, precision_recall_curve,
                             precision_recall_fscore_support, f1_score,
                             precision_recall_fscore_support)

# Configuration
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)


# Supress warnings
import warnings
warnings.filterwarnings("ignore")

print("Setup complete...")

Setup complete...


### 1.2. Common Function

In [296]:
# Distribution plot

def analyse_continuous(df,var,target):
    df = df.copy()
    # df[var] = df[var].fillna(df[var].median())
    plt.figure(figsize=(20,5))
       
    # histogram
    plt.subplot(131)
    sns.distplot(df[var], bins=30)
    #sns.distplot(df[var],hist=True, kde=True,kde_kws={'shade': True, 'linewidth': 3})
    plt.title('Histogram')    
    
    # Q-Q plot
    plt.subplot(132)
    stats.probplot(df[var], dist="norm", plot=pylab)
    plt.ylabel('Quantiles')    
    
    # boxplot
    plt.subplot(133)
    sns.boxplot(x=df[var])
    plt.title('Boxplot')
          
    # skewness and kurtosis
    print('Skewness: %f' % df[var].skew())
    print('Kurtosis: %f' % df[var].kurt())
    plt.show()

In [297]:
def Training_Preparation(df, cont_vars):
    num_df = df[cont_vars].copy()

    # scaling features
    from sklearn.preprocessing import MinMaxScaler
    numdf_norm = pd.DataFrame(MinMaxScaler().fit_transform(df[cont_vars]))
    numdf_norm.columns = num_df.columns
    
    # Define X & y
    X = numdf_norm
    y = df['Class']

    # Split to train and test set
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=90, stratify = y)
    
    # initialize models
    models = []
    models.append(('KNN', KNeighborsClassifier()))
    models.append(('SVC', SVC(kernel="linear")))
    models.append(('LSVC', SVC(kernel="rbf")))
    models.append(('LR', LogisticRegression()))
    models.append(('DT', DecisionTreeClassifier()))
    models.append(('GNB', GaussianNB()))
    models.append(('RF', RandomForestClassifier()))
    models.append(('GB', GradientBoostingClassifier()))
    models.append(('LGB',LGBMClassifier()))
    models.append(('ADA',AdaBoostClassifier()))
    models.append(('LDA',LinearDiscriminantAnalysis()))
    models.append(('QDA',QuadraticDiscriminantAnalysis()))
    models.append(('NN',MLPClassifier()))
    models.append(('XGB',XGBClassifier()))
    
    # Test options and evaluation metric
    seed = 9
    scoring = 'recall_macro'

    # evaluate each model in turn
    results = {}
    names = []

    for name, model in models:
        kfold = model_selection.KFold(n_splits=10, random_state = seed)
        cv_results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
        results[name] = cv_results
        names.append(name)
        msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
        print(msg)
        
    results_df = pd.DataFrame(results)
    plt.figure(figsize=(16,8))
    sns.boxplot(data=results_df)
    plt.show()

In [298]:
from matplotlib.backends.backend_pdf import PdfPages


def DistributionComparison(all_df, selected_vars,name):
    colors = ['#3791D7','#D72626']

    # pdf = matplotlib.backends.backend_pdf.PdfPages(name + '.pdf')
    with PdfPages(name + '.pdf') as pdf_pages:
        for column in selected_vars:    
            fig = plt.figure(figsize=[8,4])
            plt.subplot(121)
            sns.boxplot(x='Class', y=column,data=all_df,palette=colors)
            plt.title(column, fontsize=12)
            plt.subplot(122)
            sns.kdeplot(all_df[all_df.Class==1][column], bw = 0.4, label = "Cancer", shade=True, color="#D72626", linestyle="--")
            sns.kdeplot(all_df[all_df.Class==0][column], bw = 0.4, label = "NoCancer", shade=True, color= "#3791D7", linestyle=":")
            plt.title(column, fontsize=12)   
            pdf_pages.savefig(fig)                                          
            plt.show()    

    # Write the PDF document to the disk
    #pdf_pages.close()

In [299]:
def ModelEvaluation(df, cont_vars):
    
    num_df = df[cont_vars].copy()

    # scaling features
    from sklearn.preprocessing import MinMaxScaler
    numdf_norm = pd.DataFrame(MinMaxScaler().fit_transform(df[cont_vars]))
    numdf_norm.columns = num_df.columns
    
    # Define X & y
    X = numdf_norm
    y = df['Class']

    # Split to train and test set
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=90, stratify = y)
    
    # initialize models
    models = []
    models.append(('KNN', KNeighborsClassifier()))
    models.append(('SVC', SVC(kernel="linear")))
    models.append(('LSVC', SVC(kernel="rbf")))
    models.append(('LR', LogisticRegression()))
    models.append(('DT', DecisionTreeClassifier()))
    models.append(('GNB', GaussianNB()))
    models.append(('RF', RandomForestClassifier()))
    models.append(('GB', GradientBoostingClassifier()))
    models.append(('LGB',LGBMClassifier()))
    models.append(('ADA',AdaBoostClassifier()))
    models.append(('LDA',LinearDiscriminantAnalysis()))
    models.append(('QDA',QuadraticDiscriminantAnalysis()))
    models.append(('NN',MLPClassifier()))
    models.append(('XGB',XGBClassifier()))
    
    for name,model in models:
        print(name)
        model.fit(X_train, y_train)
        
        print('==========================================================')
        print('Train set')
        y_train_pred = model.predict(X_train)
        print('Accuracy: ', accuracy_score(y_train, list(y_train_pred)))
        print('ROC AUC Score: ', roc_auc_score(y_train, list(y_train_pred)))
        cm_df = pd.DataFrame(confusion_matrix(y_train,list(y_train_pred)), index=model.classes_,columns=model.classes_)
        cm_df.index.name = 'True'
        cm_df.columns.name = 'Predicted'
        print('Confusion matrix')
        print(cm_df)
        print(classification_report(y_train, list(y_train_pred)))
  
        print('----------------------------------------------------------')
        print('Test set')
        y_test_pred = model.predict(X_test)
        print('Accuracy: ', accuracy_score(y_test, list(y_test_pred)))
        print('ROC AUC Score: ', roc_auc_score(y_test, list(y_test_pred)))
        cm_df = pd.DataFrame(confusion_matrix(y_test,list(y_test_pred)), index=model.classes_,columns=model.classes_)
        cm_df.index.name = 'True'
        cm_df.columns.name = 'Predicted'
        print('Confusion matrix')
        print(cm_df)
        print(classification_report(y_test, list(y_test_pred)))
        print('==========================================================')
        

In [300]:
from sklearn.manifold import TSNE

def tsne_plot(X, y):
       
        
    # scaling features
    from sklearn.preprocessing import MinMaxScaler
    numdf_norm = pd.DataFrame(MinMaxScaler().fit_transform(X))
    numdf_norm.columns = X.columns
    
    tsne = TSNE(n_components=2, random_state=0)
    X_t = tsne.fit_transform(numdf_norm)

    plt.figure(figsize=(12, 8))
    plt.scatter(X_t[np.where(y == 0), 0], X_t[np.where(y == 0), 1], marker='o', color='g', linewidth='1', alpha=0.8, label='No cancer')
    plt.scatter(X_t[np.where(y == 1), 0], X_t[np.where(y == 1), 1], marker='o', color='r', linewidth='1', alpha=0.8, label='Colon cancer')

    plt.legend(loc='best');
    plt.show();

In [301]:
# function to find upper and lower boundaries
# for normally distributed variables

def find_normal_boundaries(df, variable):

    # calculate the boundaries outside which sit the outliers
    # for a Gaussian distribution

    upper_boundary = df[variable].mean() + 3 * df[variable].std()
    lower_boundary = df[variable].mean() - 3 * df[variable].std()

    return upper_boundary, lower_boundary

In [302]:
# function to find upper and lower boundaries
# for skewed distributed variables

def find_skewed_boundaries(df, variable, distance):

    # Let's calculate the boundaries outside which sit the outliers
    # for skewed distributions

    # distance passed as an argument, gives us the option to
    # estimate 1.5 times or 3 times the IQR to calculate
    # the boundaries.

    IQR = df[variable].quantile(0.75) - df[variable].quantile(0.25)

    lower_boundary = df[variable].quantile(0.25) - (IQR * distance)
    upper_boundary = df[variable].quantile(0.75) + (IQR * distance)

    return upper_boundary, lower_boundary

In [303]:
def find_uncorrelated_vars(cancer_df, selected_vars, threshold):

    corrmat = cancer_df[selected_vars].corr()
    corrmat = corrmat.abs().unstack() # absolute value of corr coef
    corrmat = corrmat.sort_values(ascending=False)

    corrmat = pd.DataFrame(corrmat).reset_index()
    corrmat.columns = ['feature1', 'feature2', 'corr']
    corrmat['MissingF1'] = corrmat.feature1.apply(lambda x:MissingPercentage(x))
    corrmat['MissingF2'] = corrmat.feature2.apply(lambda x:MissingPercentage(x))
    
    correlated_groups = corrmat[corrmat['corr'] > threshold]
    
    selected_vars = []
    remaining_vars = correlated_groups.feature1.unique()

    while(len(remaining_vars) > 0):
        feature = remaining_vars[0]
        correlated_block = correlated_groups[correlated_groups.feature1 == feature]
        min_ind = correlated_block[['MissingF2']].idxmin() 
        sel_var = correlated_block.feature2[min_ind].values[0]
        removed_vars = [var for var in list(correlated_block.feature2.values)]
        remaining_vars = [var for var in remaining_vars if var not in removed_vars]
        if sel_var not in selected_vars:
            selected_vars = selected_vars + [sel_var]   
    
    return selected_vars

In [304]:
def analyze_na_values(df, var, target):
    tmp_df = df.copy()
    print(target)
    
    # Make a variable that indicates 1 if the observation was missing or 0 otherwise
    tmp_df['Missing'] = np.where(df[var].isnull(),1,0)
    
    # Calculate the mean Price where the information is missing or present
    tmp_df =  pd.DataFrame(tmp_df.groupby([target,'Missing'])[target].count())
    tmp_df.columns = ['Count']
    tmp_df = tmp_df.reset_index()
    
    
    if(len(tmp_df[tmp_df[target] == 0] == 1)):
        tmp_df= tmp_df.append({target:0,'Missing':1,'Count':0}, ignore_index=True)

    if(len(tmp_df[tmp_df[target] == 1] == 1)):
        tmp_df= tmp_df.append({target:1,'Missing':1,'Count':0}, ignore_index=True)

    tmp_df.loc[0,'Per']= tmp_df.loc[0,'Count']/(tmp_df.loc[0:1,'Count'].sum())
    tmp_df.loc[1,'Per']= tmp_df.loc[1,'Count']/(tmp_df.loc[0:1,'Count'].sum())
    tmp_df.loc[2,'Per']= tmp_df.loc[2,'Count']/(tmp_df.loc[2:3,'Count'].sum())
    tmp_df.loc[3,'Per']= tmp_df.loc[3,'Count']/(tmp_df.loc[2:3,'Count'].sum())
    sns.barplot(x=target, y = 'Per', data=tmp_df, hue='Missing')
    plt.title(var)
    plt.show()

In [305]:
def CategoricalDistribution(df, var, target):
    df = df.copy()
    
    # Calculate the mean Price where the information is missing or present
    sns.countplot(x=var, data=df, hue=target)
    plt.title(var)
    plt.show()

In [306]:
def CreateDummyVar(df, categorical_list):
    objdf_new = df.copy()
    objdf_dummy =pd.DataFrame()
    i = 0
    for e in categorical_list:
        i = i + 1
        objdf_new[e] = e + '_' + objdf_new[e].astype(str)
        varname= e 
        df_temp = pd.get_dummies(objdf_new[varname], drop_first=True)
        objdf_dummy = pd.concat([objdf_dummy, df_temp], axis=1)
        
    return objdf_dummy

In [307]:
def MissingPercentage(x):
    return df[x].isnull().sum()/len(df)

## 2. Load raw data

In [308]:
df = pd.read_csv('Nhanes-BreastCancer_merged.csv')

df.shape

(1173, 461)

In [310]:
df.head()

,RIDAGEYR,RIDAGEMN,INDFMPIR,LBXSAL,LBDSGLSI,URDTIME2,LBDSBUSI,LBDHDDSI,LBDLYMNO,LBXSCA,URXMBP,LBDSCASI,URXMZP,URDFLOW2,LBXGLU,LBXBGE,LBXBPB,LBDBCDSI,URXUMS,LBXSGL,LBXSOSSI,LBXIHG,URXMHH,URXVOL3,URXMHP,URDTIME1,LBXSZN,LBXSKSI,LBDTHGSI,URXMIB,URDFLOW3,URXMNP,LBXBCD,LBXSATSI,LBXSBU,LBDSTRSI,LBDSCRSI,LBXSUA,LBDSTPSI,LBXSTR,URXVOL1,LBXLYPCT,URXVOL2,LBDSTBSI,URXECP,LBDTCSI,LBXNEPCT,LBXBGM,LBXHGB,LBXSTP,URXCNP,LBXPLTSI,LBDIHGSI,LBDSIRSI,LBXSGTSI,LBXBAPCT,LBXSTB,LBXMCHSI,LBDNENO,LBXSCR,LBDSZNSI,LBXMOPCT,LBXMPSI,LBXSIR,PHAANTMN,LBDSCUSI,PHAALCMN,LBXRBCSI,URXUIO,LBDSUASI,LBDSPHSI,URXCRS,URXMEP,LBXMCVSI,LBXSCH,URXMC1,LBXGLT,WTSVS2YR,LBXSLDSI,LBXRDW,WTFSM,PHAGUMMN,LBXSGB,LBDBPBSI,LBDGLTSI,WTSVOC2Y,URXCOP,LBDGLUSI,LBXSCU,LBXSSE,LBXWBCSI,LBDHDD,LBDSSESI,URDFLOW1,LBXSPH,PHACOFMN,PHASUPMN,WTSOG2YR,URXUMA,LBXSASSI,LBXEOPCT,LBDSGBSI,LBXSAPSI,LBXTC,LBXMC,URDTIME3,LBXTHG,LBDSCHSI,URXMOH,LBDMONO,LBDEONO,LBDBANO,LBDSALSI,PHASUPHR,LBXSCLSI,LBXSNASI,PHAGUMHR,PHAANTHR,PHACOFHR,PHAALCHR,LBXSC3SI,DXDLAPF,DXDTRBMD,DXDSTBMC,DXXTRFAT,DXXHEFAT,DXDSTLE,DXXRALI,DXDTOFAT,DXDTOBMD,DXXLSBMC,DXXTRLI,DXDRATOT,BPXDI3,DXXHEA,DXDTOPF,DXDRLTOT,DXDTRPF,DXXLLBMD,DXXRLLI,BMXARMC,DXDRALE,DXXLLBMC,DXXLALI,DXXLRBMC,DXXPEA,DXDRAPF,BPXDI1,DXXPEBMD,DXDTRTOT,DXXRAA,DXDLLPF,BMXARML,DXXHELI,DXXRRA,DXDSTBMD,BPXDI4,DXXLLLI,DXDRLPF,BPXDI2,DXXLSBMD,BPXPLS,DXDTOBMC,DXDTOLE,DXXLRBMD,DXDTRLE,DXXHEBMC,DXDSTTOT,DXXRLA,BMXLEG,BPXSY1,DXXLAA,DXXLABMC,DXDLALE,DXDTRA,BMXWT,DXXTSBMC,DXXLLA,DXXLLFAT,DXDTOLI,BMXHT,DXXLABMD,DXDSTLI,DXXRRBMD,DXXLSA,DXDSTA,DXDTRBMC,DXXRABMC,DXDHELE,DXDHETOT,DXXHEBMD,DXDTOA,DXXRABMD,DXDRLLE,DXXLRA,DXXRRBMC,BPXSY2,DXDLLLE,DXXTSBMD,BMXBMI,DXDSTFAT,DXXRLFAT,DXXRLBMD,DXXRAFAT,DXDLATOT,BMXWAIST,DXDLLTOT,DXDSTPF,DXDHEPF,DXXLAFAT,DXXRLBMC,BPXSY3,BPXSY4,DXXTSA,DXXPEBMC,DXDTOTOT,DMDYRSUS,DMDEDUC2,DMDHHSIZ,DMDFMSIZ,BPXML1,BPACSZ,FCX10DI,FCX11DI,FCX06DI,FCX08DI,FCX07DI,FCX09DI,RIDRETH1,RIDEXMON,DMDCITZN,DMDMARTL,RIDEXPRG,URDMNPLC,PHQ060,ORXH51,ORXH64,ORXHPC,LBDHEG,LBDWFL,ORXGL,ORXH69,PHQ050,LBDHD,LBXHCR,ORXH62,URDCNPLC,ORXHPV,ORXH11,ORXH73,URXUTRI,ORXH26,ORXH31,LBDHBG,ORXH82,LBXHE1,ORXH53,ORXH58,LBDHEM,URDMZPLC,PHQ020,ORXH35,ORXH84,LBDIHGLC,URXPREG,PHQ040,LBDBGELC,ORXH68,URDMEPLC,ORXH81,ORXGH,ORXH52,URDMBPLC,ORXH56,LBXHE2,ORXH61,LBDBGMLC,URDECPLC,URDMOHLC,ORXH33,ORXH06,ORXHPI,LBDTHGLC,URDMHPLC,URDCOPLC,URDMC1LC,ORXH45,ORXH55,ORXH71,PHQ030,ORXH40,ORXH39,ORXH83,ORXH66,LBXHBC,ORXH72,ORXH54,ORXH70,URXUCL,LBDWFLLC,LBXHA,ORXH59,ORXH67,ORXH42,ORXH18,URDMHHLC,PHDSESN,URDMIBLC,ORXH16,LBXHBS,LBXHCG,OHARNF,OHAPOS,BMIARML,OHDDESTS,OHAROCGP,OHX23TC,BPAEN2,OHAROCOH,OHAROCDE,OHX02TC,OHX30TC,OHXIMP,OHX14TC,OHAROTH,BMDSTATS,OHX16TC,OHX05TC,BMIARMC,BPAEN3,OHX26TC,OHAREC,BPAARM,OHAROCCI,BMILEG,BPAEN4,OHX01TC,BMXRECUM,OHX09TC,BMIHT,BPXCHR,OHX32TC,OHX18TC,OHDRCSTS,BPXPTY,BMXHEAD,OHX19TC,OHX15TC,OHAROCDT,BMIHEAD,BMIRECUM,OHX03TC,BPXPULS,OHX17TC,BMIWT,BPAEN1,OHX31TC,OHX08TC,BMIWAIST,DXARLBV,DXALLBV,DXARABV,OHX06TC,DXALABV,DXARLTV,OHX04TC,OHX12TC,OHX27TC,OHX13TC,DXAHEBV,OHX22TC,OHX10TC,OHX29TC,OHX21TC,OHX28TC,OHX07TC,DXARATV,OHX24TC,PEASCCT1,DXALLTV,OHX25TC,OHX11TC,DXAHETV,DXAEXSTS,LBD2DFLC,LBDV1DLC,LBDV2ALC,LBDV3BLC,LBDV4CLC,LBDVBZLC,LBDVCBLC,LBDVCTLC,LBDVDBLC,LBDVEBLC,LBDVMCLC,LBDVNBLC,LBDVOXLC,LBDVTCLC,LBDVTELC,LBDVXYLC,LBX2DF,LBXV1D,LBXV2A,LBXV3B,LBXV4C,LBXVBZ,LBXVCB,LBXVCT,LBXVDB,LBXVEB,LBXVMC,LBXVNB,LBXVOX,LBXVTC,LBXVTE,LBXVXY,OHDEXSTS,PHAFSTHR,PHAFSTMN,URDUA3LC,URDUA5LC,URDUABLC,URDUACLC,URDUBALC,URDUCDLC,URDUCOLC,URDUCSLC,URDUDALC,URDUMMAL,URDUMNLC,URDUMOLC,URDUPBLC,URDUSBLC,URDUSNLC,URDUSRLC,URDUTLLC,URDUTULC,URDUURLC,URXUAB,URXUAC,URXUAS,URXUAS3,URXUAS5,URXUBA,URXUCD,URXUCO,URXUCS,URXUDMA,URXUMMA,URXUMN,URXUMO,URXUPB,URXUSB,URXUSN,URXUSR,URXUTL,URXUTU,URXUUR,WTSA2YR,WTSAF2YR,Class
0,72,NaN,5.00,3.7,4.94,NaN,4.64,1.81,1.9,8.9,NaN,2.225,NaN,NaN,NaN,0.11,1.83,6.58,1.5,89.0,275.0,0.46,NaN,NaN,NaN,57.0,NaN,3.9,25.9,NaN,NaN,NaN,0.74,18.0,13.0,1.332,67.18,5.2,69.0,118.0,76.0,33.4,NaN,8.55,NaN,5.17,53.4,5.09,12.4,6.9,NaN,237.0,2.30,11.1,20.0,0.6,0.5,31.2,3.0,0.76,NaN,8.1,9.0,62.0,NaN,NaN,NaN,3.96,Na

## 4. Target variable analysis

In [311]:
df[df.Class == 1].shape[0]/df.shape[0],df[df.Class == 1].shape[0],df[df.Class == 0].shape[0]

(0.4919011082693947, 577, 596)

## 5. Categorize vars

In [312]:
target = ['Class']

cont_vars = ['RIDAGEYR', 'RIDAGEMN', 'INDFMPIR', 'LBXSAL', 'LBDSGLSI', 'URDTIME2', 'LBDSBUSI', 'LBDHDDSI', 'LBDLYMNO', 'LBXSCA', 'URXMBP', 'LBDSCASI', 'URXMZP', 'URDFLOW2', 'LBXGLU', 'LBXBPB', 'LBDBCDSI', 'URXUMS', 'LBXSGL', 'LBXSOSSI', 'URXUAC', 'LBXIHG', 'URXMHH', 'URXMHP', 'URXUBA', 'WTFSM', 'URXUUR', 'URDTIME1', 'URXUSR', 'LBXSZN', 'URXUTU', 'LBXSKSI', 'URXUSN', 'URXUMMA', 'LBDTHGSI', 'URXMIB', 'URXUMO', 'URXMNP', 'LBXBCD', 'URXUDMA', 'LBXSATSI', 'LBXSBU', 'LBXV4C', 'LBDSTRSI', 'LBXVBZ', 'LBDSCRSI', 'LBXSUA', 'LBXVOX', 'URXUAS', 'URXUSB', 'LBDSTPSI', 'LBXVXY', 'LBXSTR', 'WTSA2YR', 'URXVOL1', 'LBXLYPCT', 'URXUTL', 'URXUPB', 'URXVOL2', 'PHAFSTMN', 'LBXVEB', 'URXECP', 'LBDTCSI', 'LBXNEPCT', 'LBXBGM', 'LBXHGB', 'LBXSTP', 'URXCNP', 'LBXPLTSI', 'LBDIHGSI', 'LBDSIRSI', 'LBXSGTSI', 'URXUCO', 'LBXBAPCT', 'LBXMCHSI', 'LBDNENO', 'LBXSCR', 'LBDSZNSI', 'LBXMOPCT', 'LBXMPSI', 'LBXVDB', 'LBXSIR', 'URXUCS', 'LBX2DF', 'URXUAB', 'LBDSCUSI', 'LBXRBCSI', 'URXUIO', 'LBDSUASI', 'LBDSPHSI', 'URXCRS', 'URXMEP', 'URXUCD', 'LBXMCVSI', 'URXUAS3', 'LBXSCH', 'URXMC1', 'LBXGLT', 'WTSVS2YR', 'LBXSLDSI', 'LBXRDW', 'PHAGUMMN', 'LBXSGB', 'LBDBPBSI', 'LBDGLTSI', 'WTSVOC2Y', 'URXCOP', 'LBDGLUSI', 'LBXSCU', 'LBXSSE', 'LBXWBCSI', 'LBDHDD', 'LBDSSESI', 'URDFLOW1', 'WTSAF2YR', 'LBXSPH', 'WTSOG2YR', 'URXUMA', 'LBXSASSI', 'LBXEOPCT', 'LBDSGBSI', 'LBXSAPSI', 'LBXTC', 'LBXMC', 'LBXTHG', 'LBDSCHSI', 'URXMOH', 'LBDSALSI', 'PHAFSTHR', 'DXDLAPF', 'DXDTRBMD', 'DXDSTBMC', 'DXXTRFAT', 'DXXHEFAT', 'DXDSTLE', 'DXXRALI', 'DXDTOFAT', 'DXDTOBMD', 'DXXLSBMC', 'DXXTRLI', 'DXDRATOT', 'BPXDI3', 'DXXHEA', 'DXDTOPF', 'DXDRLTOT', 'DXDTRPF', 'DXXLLBMD', 'DXXRLLI', 'BMXARMC', 'DXDRALE', 'DXXLLBMC', 'DXXLALI', 'DXXLRBMC', 'DXXPEA', 'DXDRAPF', 'BPXDI1', 'DXXPEBMD', 'DXDTRTOT', 'DXXRAA', 'DXDLLPF', 'BMXARML', 'DXXHELI', 'DXXRRA', 'DXDSTBMD', 'BPXDI4', 'DXXLLLI', 'DXDRLPF', 'BPXDI2', 'DXXLSBMD', 'BPXPLS', 'DXDTOBMC', 'DXDTOLE', 'DXXLRBMD', 'DXDTRLE', 'DXXHEBMC', 'DXDSTTOT', 'DXXRLA', 'BMXLEG', 'BPXSY1', 'DXXLAA', 'DXXLABMC', 'DXDLALE', 'DXDTRA', 'BMXWT', 'DXXTSBMC', 'DXXLLA', 'DXXLLFAT', 'DXDTOLI', 'BMXHT', 'DXXLABMD', 'DXDSTLI', 'DXXRRBMD', 'DXXLSA', 'DXDSTA', 'DXDTRBMC', 'DXXRABMC', 'DXDHELE', 'DXDHETOT', 'DXXHEBMD', 'DXDTOA', 'DXXRABMD', 'DXDRLLE', 'DXXLRA', 'DXXRRBMC', 'BPXSY2', 'DXDLLLE', 'DXXTSBMD', 'BMXBMI', 'DXDSTFAT', 'DXXRLFAT', 'DXXRLBMD', 'DXXRAFAT', 'DXDLATOT', 'BMXWAIST', 'DXDLLTOT', 'DXDSTPF', 'DXDHEPF', 'DXXLAFAT', 'DXXRLBMC', 'BPXSY3', 'BPXSY4', 'DXXTSA', 'DXXPEBMC', 'DXDTOTOT']
print(len(cont_vars))

dis_vars = ['DMDYRSUS', 'DMDEDUC2', 'DMDHHSIZ', 'DMDFMSIZ', 'BPXML1', 'BPACSZ', 'FCX10DI', 'FCX11DI', 'FCX06DI', 'FCX08DI', 'FCX07DI', 'FCX09DI', 'LBXBGE', 'URXVOL3', 'URXUAS5', 'URDFLOW3', 'LBXVTE', 'LBDSTBSI', 'URXUMN', 'LBXVTC', 'LBXSTB', 'PHAANTMN', 'PHAALCMN', 'PHACOFMN', 'PHASUPMN', 'URDTIME3', 'LBXV2A', 'LBXV1D', 'LBXVMC', 'LBXVNB', 'LBXV3B', 'LBXVCB', 'LBXVCT', 'LBDMONO', 'LBDEONO', 'LBDBANO', 'PHASUPHR', 'LBXSCLSI', 'LBXSNASI', 'PHAGUMHR', 'PHAANTHR', 'PHACOFHR', 'PHAALCHR', 'LBXSC3SI']
print(len(dis_vars))

cat_vars = ['RIDRETH1', 'RIDEXMON', 'DMDCITZN', 'DMDMARTL', 'RIDEXPRG', 'URDMNPLC', 'PHQ060', 'ORXH51', 'ORXH64', 'ORXHPC', 'LBDHEG', 'ORXGL', 'ORXH69', 'PHQ050', 'LBDVCTLC', 'LBD2DFLC', 'LBXHCR', 'ORXH62', 'URDCNPLC', 'URDUSNLC', 'ORXHPV', 'LBDVTCLC', 'URDUA3LC', 'URDUTLLC', 'URDUTULC', 'URDUDALC', 'ORXH11', 'URDUCOLC', 'ORXH73', 'URXUTRI', 'ORXH26', 'ORXH31', 'LBDHBG', 'ORXH82', 'LBXHE1', 'ORXH53', 'ORXH58', 'URDUURLC', 'LBDHEM', 'URDMZPLC', 'URDUBALC', 'LBDVTELC', 'LBDV1DLC', 'PHQ020', 'LBDV3BLC', 'URDUACLC', 'URDUSBLC', 'ORXH35', 'ORXH84', 'LBDIHGLC', 'URXPREG', 'LBDV4CLC', 'PHQ040', 'URDUPBLC', 'LBDBGELC', 'URDUMMAL', 'URDUMNLC', 'ORXH68', 'URDUA5LC', 'URDMEPLC', 'ORXH81', 'LBDVOXLC', 'ORXGH', 'LBDVCBLC', 'LBDVBZLC', 'ORXH52', 'URDMBPLC', 'ORXH56', 'LBXHE2', 'ORXH61', 'LBDBGMLC', 'URDECPLC', 'URDUCDLC', 'LBDVNBLC', 'URDMOHLC', 'ORXH33', 'ORXH06', 'ORXHPI', 'LBDTHGLC', 'URDUMOLC', 'LBDVXYLC', 'URDMHPLC', 'URDCOPLC', 'URDMC1LC', 'ORXH45', 'ORXH55', 'ORXH71', 'PHQ030', 'LBDVMCLC', 'URDUSRLC', 'ORXH40', 'ORXH39', 'ORXH83', 'ORXH66', 'LBXHBC', 'LBDVDBLC', 'LBDVEBLC', 'ORXH72', 'ORXH54', 'ORXH70', 'URDUCSLC', 'URXUCL', 'LBDV2ALC', 'LBDWFLLC', 'URDUABLC', 'LBXHA', 'ORXH59', 'ORXH67', 'ORXH42', 'ORXH18', 'URDMHHLC', 'PHDSESN', 'URDMIBLC', 'ORXH16', 'LBXHBS', 'LBXHCG', 'OHARNF', 'OHAPOS', 'BMIARML', 'OHDDESTS', 'OHAROCGP', 'OHX23TC', 'BPAEN2', 'OHAROCOH', 'OHAROCDE', 'OHX02TC', 'OHX30TC', 'OHXIMP', 'OHX14TC', 'OHAROTH', 'BMDSTATS', 'OHX16TC', 'OHX05TC', 'BMIARMC', 'BPAEN3', 'OHX26TC', 'OHAREC', 'BPAARM', 'OHAROCCI', 'BMILEG', 'BPAEN4', 'OHX01TC', 'BMXRECUM', 'OHX09TC', 'BMIHT', 'BPXCHR', 'OHX32TC', 'OHX18TC', 'OHDRCSTS', 'BPXPTY', 'BMXHEAD', 'OHX19TC', 'OHX15TC', 'OHDEXSTS', 'OHAROCDT', 'BMIHEAD', 'BMIRECUM', 'OHX03TC', 'BPXPULS', 'OHX17TC', 'BMIWT', 'BPAEN1', 'OHX31TC', 'OHX08TC', 'BMIWAIST', 'DXARLBV', 'DXALLBV', 'DXARABV', 'FCX10DI', 'OHX06TC', 'DXALABV', 'DXARLTV', 'OHX04TC', 'OHX12TC', 'FCX08DI', 'OHX27TC', 'OHX13TC', 'DXAHEBV', 'OHX22TC', 'OHX10TC', 'OHX29TC', 'OHX21TC', 'FCX07DI', 'OHX28TC', 'OHX07TC', 'FCX09DI', 'DXARATV', 'OHX24TC', 'PEASCCT1', 'DXALLTV', 'OHX25TC', 'OHX11TC', 'DXAHETV', 'DXAEXSTS']      
print(len(cat_vars))

224
44
194


## 6. Final dataset

In [313]:
all_df = df.copy()
all_df.shape

(1173, 461)

## 7. Check missing data of all features

In [314]:
all_vars = cont_vars+dis_vars+cat_vars
miss_df = pd.DataFrame(all_df[all_vars].isnull().sum(),columns=['Count'])
miss_df['Percentage'] = 100 * all_df[all_vars].isnull().sum()/len(all_df)
miss_df = miss_df.sort_values('Percentage', ascending=True)
miss_df = miss_df.reset_index()
miss_df.columns = ['Feature','Count','Percentage']
miss_df.head()

,Feature,Count,Percentage
0,RIDAGEYR,0,0.0
1,DMDMARTL,0,0.0
2,DMDEDUC2,0,0.0
3,DMDHHSIZ,0,0.0
4,DMDFMSIZ,0,0.0


https://pypi.org/project/mixed-naive-bayes/

## 8. Feature scaling

In [315]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (0, 1))

X_scaled = pd.DataFrame(scaler.fit_transform(all_df[cont_vars + dis_vars]))
X_scaled.columns = cont_vars + dis_vars
print(X_scaled.shape)
X_scaled.head()

(1173, 268)


,RIDAGEYR,RIDAGEMN,INDFMPIR,LBXSAL,LBDSGLSI,URDTIME2,LBDSBUSI,LBDHDDSI,LBDLYMNO,LBXSCA,URXMBP,LBDSCASI,URXMZP,URDFLOW2,LBXGLU,LBXBPB,LBDBCDSI,URXUMS,LBXSGL,LBXSOSSI,URXUAC,LBXIHG,URXMHH,URXMHP,URXUBA,WTFSM,URXUUR,URDTIME1,URXUSR,LBXSZN,URXUTU,LBXSKSI,URXUSN,URXUMMA,LBDTHGSI,URXMIB,URXUMO,URXMNP,LBXBCD,URXUDMA,LBXSATSI,LBXSBU,LBXV4C,LBDSTRSI,LBXVBZ,LBDSCRSI,LBXSUA,LBXVOX,URXUAS,URXUSB,LBDSTPSI,LBXVXY,LBXSTR,WTSA2YR,URXVOL1,LBXLYPCT,URXUTL,URXUPB,URXVOL2,PHAFSTMN,LBXVEB,URXECP,LBDTCSI,LBXNEPCT,LBXBGM,LBXHGB,LBXSTP,URXCNP,LBXPLTSI,LBDIHGSI,LBDSIRSI,LBXSGTSI,URXUCO,LBXBAPCT,LBXMCHSI,LBDNENO,LBXSCR,LBDSZNSI,LBXMOPCT,LBXMPSI,LBXVDB,LBXSIR,URXUCS,LBX2DF,URXUAB,LBDSCUSI,LBXRBCSI,URXUIO,LBDSUASI,LBDSPHSI,URXCRS,URXMEP,URXUCD,LBXMCVSI,URXUAS3,LBXSCH,URXMC1,LBXGLT,WTSVS2YR,LBXSLDSI,LBXRDW,PHAGUMMN,LBXSGB,LBDBPBSI,LBDGLTSI,WTSVOC2Y,URXCOP,LBDGLUSI,LBXSCU,LBXSSE,LBXWBCSI,LBDHDD,LBDSSESI,URDFLOW1,WTSAF2YR,LBXSPH,WTSOG2YR,URXUMA,LBXSASSI,LBXEOPCT,LBDSGBSI,LBXSAPSI,LBXTC,LBXMC,LBXTHG,LBDSCHSI,URXMOH,LBDSALSI,PHAFSTHR,DXDLAPF,DXDTRBMD,DXDSTBMC,DXXTRFAT,DXXHEFAT,DXDSTLE,DXXRALI,DXDTOFAT,DXDTOBMD,DXXLSBMC,DXXTRLI,DXDRATOT,BPXDI3,DXXHEA,DXDTOPF,DXDRLTOT,DXDTRPF,DXXLLBMD,DXXRLLI,BMXARMC,DXDRALE,DXXLLBMC,DXXLALI,DXXLRBMC,DXXPEA,DXDRAPF,BPXDI1,DXXPEBMD,DXDTRTOT,DXXRAA,DXDLLPF,BMXARML,DXXHELI,DXXRRA,DXDSTBMD,BPXDI4,DXXLLLI,DXDRLPF,BPXDI2,DXXLSBMD,BPXPLS,DXDTOBMC,DXDTOLE,DXXLRBMD,DXDTRLE,DXXHEBMC,DXDSTTOT,DXXRLA,BMXLEG,BPXSY1,DXXLAA,DXXLABMC,DXDLALE,DXDTRA,BMXWT,DXXTSBMC,DXXLLA,DXXLLFAT,DXDTOLI,BMXHT,DXXLABMD,DXDSTLI,DXXRRBMD,DXXLSA,DXDSTA,DXDTRBMC,DXXRABMC,DXDHELE,DXDHETOT,DXXHEBMD,DXDTOA,DXXRABMD,DXDRLLE,DXXLRA,DXXRRBMC,BPXSY2,DXDLLLE,DXXTSBMD,BMXBMI,DXDSTFAT,DXXRLFAT,DXXRLBMD,DXXRAFAT,DXDLATOT,BMXWAIST,DXDLLTOT,DXDSTPF,DXDHEPF,DXXLAFAT,DXXRLBMC,BPXSY3,BPXSY4,DXXTSA,DXXPEBMC,DXDTOTOT,DMDYRSUS,DMDEDUC2,DMDHHSIZ,DMDFMSIZ,BPXML1,BPACSZ,FCX10DI,FCX11DI,FCX06DI,FCX08DI,FCX07DI,FCX09DI,LBXBGE,URXVOL3,URXUAS5,URDFLOW3,LBXVTE,LBDSTBSI,URXUMN,LBXVTC,LBXSTB,PHAANTMN,PHAALCMN,PHACOFMN,PHASUPMN,URDTIME3,LBXV2A,LBXV1D,LBXVMC,LBXVNB,LBXV3B,LBXVCB,LBXVCT,LBDMONO,LBDEONO,LBDBANO,PHASUPHR,LBXSCLSI,LBXSNASI,PHAGUMHR,PHAANTHR,PHACOFHR,PHAALCHR,LBXSC3SI
0,0.666667,NaN,1.000,0.458333,0.102870,NaN,0.138362,0.439344,0.233333,0.288889,NaN,0.288889,NaN,NaN,NaN,0.252427,0.090771,0.000336,0.102941,0.350,NaN,0.134199,NaN,NaN,NaN,NaN,NaN,0.041081,NaN,NaN,NaN,0.400000,NaN,NaN,0.230699,NaN,NaN,NaN,0.090786,NaN,0.064327,0.138462,NaN,0.115359,NaN,0.056924,0.411765,NaN,NaN,NaN,0.433333,NaN,0.115433,NaN,0.182254,0.4528,NaN,NaN,NaN,0.694915,NaN,NaN,0.293515,0.460709,0.216602,0.551020,0.433333,NaN,0.367776,0.134432,0.209026,0.027523,NaN,0.157895,0.651042,0.129252,0.056934,NaN,0.160763,0.441558,NaN,0.208511,NaN,NaN,NaN,NaN,0.423077,NaN,0.411788,0.444597,0.052838,NaN,NaN,0.571134,NaN,0.282051,NaN,NaN,NaN,0.147297,0.180000,NaN,0.363636,0.258278,NaN,NaN,NaN,NaN,NaN,NaN,0.207547,0.440678,NaN,0.180005,NaN,0.444444,NaN,0.000336,0.147059,0.306667,0.363636,0.182131,0.294118,0.676471,0.230385,0.282142,NaN,0.458333,0.038462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.653846,NaN,NaN,NaN,NaN,NaN,NaN,0.413889,NaN,NaN,NaN,NaN,NaN,NaN,0.580645,NaN,NaN,NaN,NaN,0.548387,NaN,NaN,NaN,NaN,NaN,NaN,0.587302,NaN,0.340426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.400000,0.318182,NaN,NaN,NaN,NaN,0.307393,NaN,NaN,NaN,NaN,0.488938,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.267606,NaN,NaN,0.341860,NaN,NaN,NaN,NaN,NaN,0.424865,NaN,NaN,NaN,NaN,NaN,0.250000,NaN,NaN,NaN,NaN,NaN,0.375,0.000000,0.000000,0.157658,0.75,NaN,NaN,NaN,NaN,NaN,NaN,0.230769,NaN,NaN,NaN,NaN,0.277778,NaN,NaN,0.277778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.121212,0.250000,0.0,NaN,0.526316,0.470588,NaN,NaN,NaN,NaN,0.5625
1,0.871795,NaN,0.554,0.625000,0.171302,NaN,0.169397,0.491803,0.066667,0.444444,NaN,0.444444,NaN,NaN,NaN,0.344660,0.062443,0.001877,0.171569,0.325,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.088649,NaN,NaN,NaN,0.428571,NaN,NaN,0.009991,NaN,NaN,NaN,0.062331,NaN,0.046784,0.169231,NaN,0.151812,NaN,0.077369,0.270588,NaN,NaN,Na

In [316]:
all_df[cont_vars + dis_vars] = X_scaled[cont_vars + dis_vars]

# Part 3: Data Preprocessing

## 1. Cont vars

### 1.1. Handling missing values

In [317]:
sig_cont_vars = cont_vars

miss_cont_df = pd.DataFrame(all_df[sig_cont_vars].isnull().sum(),columns=['Count'])
miss_cont_df['Percentage'] = 100 * all_df[sig_cont_vars].isnull().sum()/len(all_df)
miss_cont_df = miss_cont_df.sort_values('Percentage', ascending=False)
miss_cont_df.tail()

,Count,Percentage
BPXPLS,55,4.688832
BMXBMI,45,3.836317
BMXHT,44,3.751066
BMXWT,44,3.751066
RIDAGEYR,0,0.000000


#### Select cont vars with missing value <= 30%

In [318]:
sig_cont_vars = list(miss_cont_df[miss_cont_df.Percentage <= 30].index)
print(len(sig_cont_vars))
print(sig_cont_vars)

79
['URXVOL1', 'LBXBCD', 'LBXTHG', 'LBXBPB', 'LBDBPBSI', 'LBDTHGSI', 'LBDBCDSI', 'BPXSY3', 'BPXDI3', 'LBXSLDSI', 'LBXSASSI', 'LBXSATSI', 'LBXSOSSI', 'LBXSGL', 'LBXSAL', 'LBDSBUSI', 'LBDSPHSI', 'BPXSY2', 'LBXSIR', 'LBDSCASI', 'LBXSPH', 'LBDSUASI', 'LBXSCA', 'BPXDI2', 'LBXSBU', 'LBXSTP', 'LBDSGBSI', 'LBXSAPSI', 'LBDSCHSI', 'LBDSALSI', 'LBXSUA', 'LBDSCRSI', 'LBXSCH', 'LBDSTRSI', 'LBDSIRSI', 'LBDSTPSI', 'LBXSGTSI', 'LBXSKSI', 'LBDSGLSI', 'LBXSTR', 'LBXSCR', 'LBXSGB', 'BPXSY1', 'INDFMPIR', 'BPXDI1', 'LBDTCSI', 'LBDHDD', 'LBXTC', 'LBDHDDSI', 'LBDLYMNO', 'LBXBAPCT', 'LBDNENO', 'LBXNEPCT', 'LBXMOPCT', 'LBXEOPCT', 'LBXLYPCT', 'BMXLEG', 'LBXRDW', 'LBXHGB', 'LBXMC', 'LBXPLTSI', 'LBXWBCSI', 'LBXMCVSI', 'LBXRBCSI', 'LBXMCHSI', 'LBXMPSI', 'BMXWAIST', 'BMXARML', 'BMXARMC', 'PHAFSTHR', 'PHAFSTMN', 'URXCRS', 'URXUMS', 'URXUMA', 'BPXPLS', 'BMXBMI', 'BMXHT', 'BMXWT', 'RIDAGEYR']


#### Remove observations with 40% of missing data

In [319]:
all_df['FeatureCount'] = all_df[sig_cont_vars].count(axis=1)
all_df['FeatureMissing'] = len(sig_cont_vars) - all_df['FeatureCount']
all_df['MissingPercentage'] = all_df.FeatureMissing/len(sig_cont_vars)
all_df[['FeatureMissing','MissingPercentage']].describe()

,FeatureMissing,MissingPercentage
count,1173.000000,1173.000000
mean,7.884058,0.099798
std,17.917955,0.226810
min,0.000000,0.000000
25%,0.000000,0.000000
50%,1.000000,0.012658
75%,6.000000,0.075949
max,78.000000,0.987342


In [320]:
print(all_df[(all_df['MissingPercentage'] > 0.4) & (all_df['Class'] == 1)].shape)
all_df = all_df[all_df['MissingPercentage'] <= 0.4]
all_df = all_df.reset_index(drop=True)

all_df.shape, len(all_df[(all_df['Class'] == 1)])/len(all_df)

(92, 464)


((1054, 464), 0.4601518026565465)

#### Filling missing data 

In [321]:
for var in sig_cont_vars:
    all_df[var] = all_df[var].fillna(all_df[var].median())

In [322]:
from sklearn.naive_bayes import GaussianNB

X_train, X_test, y_train, y_test = train_test_split(all_df[sig_cont_vars], all_df['Class'], test_size=0.25, random_state=1)

def NaiveBayesPrediction(X_train, y_train, X_test, y_test):
    clf = GaussianNB()
    clf.fit(X_train, y_train)
    
    y_pred_gnb = clf.predict(X_test)
    y_prob_pred_gnb = clf.predict_proba(X_test)
    # how did our model perform?
    count_misclassified = (y_test != y_pred_gnb).sum()
    
    print("GaussianNB")
    print("=" * 30)
    print('Misclassified samples: {}'.format(count_misclassified))
    accuracy = accuracy_score(y_test, y_pred_gnb)
    print('Accuracy: {:.2f}'.format(accuracy))
    
    print("Recall score : ", recall_score(y_test, y_pred_gnb , average='micro'))
    print("Precision score : ",precision_score(y_test, y_pred_gnb , average='micro'))
    print("F1 score : ",f1_score(y_test, y_pred_gnb , average='micro'))
    
    print(classification_report(y_test, y_pred_gnb))
    
NaiveBayesPrediction(X_train, y_train, X_test, y_test)

GaussianNB
Misclassified samples: 93
Accuracy: 0.65
Recall score :  0.6477272727272727
Precision score :  0.6477272727272727
F1 score :  0.6477272727272727
              precision    recall  f1-score   support

           0       0.62      0.89      0.73       142
           1       0.75      0.36      0.49       122

    accuracy                           0.65       264
   macro avg       0.68      0.63      0.61       264
weighted avg       0.68      0.65      0.62       264



In [323]:
from sklearn.naive_bayes import GaussianNB 
model = GaussianNB()

kfold = KFold(n_splits = 10, random_state = 10)
scores = cross_val_score(model,X_train,y_train,cv=kfold,scoring='recall_macro')
print(scores)

print("Accuracy: %0.2f%% (+/- %0.2f%%)" % (100*scores.mean(), 100*scores.std()))

[0.64247312 0.58039216 0.66111111 0.59198718 0.62799202 0.6262987
 0.56915584 0.65423387 0.62612613 0.69664032]
Accuracy: 62.76% (+/- 3.71%)


### 1.2. Remove outliers

In [324]:
for var in sig_cont_vars:
    upper_boundary, lower_boundary = find_skewed_boundaries(all_df, var, 3.5)
    all_df.loc[all_df[var] <= lower_boundary,var] = lower_boundary
    all_df.loc[all_df[var] >= upper_boundary,var] = upper_boundary

In [325]:
from sklearn.naive_bayes import GaussianNB

X_train, X_test, y_train, y_test = train_test_split(all_df[sig_cont_vars], all_df['Class'], test_size=0.25, random_state=1)
    
NaiveBayesPrediction(X_train, y_train, X_test, y_test)

GaussianNB
Misclassified samples: 87
Accuracy: 0.67
Recall score :  0.6704545454545454
Precision score :  0.6704545454545454
F1 score :  0.6704545454545454
              precision    recall  f1-score   support

           0       0.67      0.77      0.72       142
           1       0.68      0.55      0.61       122

    accuracy                           0.67       264
   macro avg       0.67      0.66      0.66       264
weighted avg       0.67      0.67      0.67       264



In [326]:
from sklearn.naive_bayes import GaussianNB 
model = GaussianNB()

kfold = KFold(n_splits = 10, random_state = 10)
scores = cross_val_score(model,X_train,y_train,cv=kfold,scoring='recall_macro')
print(scores)

print("Accuracy: %0.2f%% (+/- %0.2f%%)" % (100*scores.mean(), 100*scores.std()))

[0.69758065 0.70522876 0.77124183 0.63108974 0.67420213 0.64642857
 0.63506494 0.75268817 0.67052767 0.71805007]
Accuracy: 69.02% (+/- 4.54%)


### 1.3. Gaussian Transformation - Type 2

In [327]:
df1 = all_df.copy()

Gauss_transformed_vars = ['URDTIME1','URXVOL1','LBXTHG','LBDTHGSI','LBXSGTSI','URXCRS','URXUMS','URXUMA']

In [328]:
from sklearn.preprocessing import quantile_transform

for var in Gauss_transformed_vars:
    df1[var] = quantile_transform(np.array(df1[var]).reshape(-1,1), n_quantiles=20, random_state=0, copy=True)

In [329]:
X_train, X_test, y_train, y_test = train_test_split(df1[sig_cont_vars], df1['Class'], test_size=0.25, random_state=1)

NaiveBayesPrediction(X_train, y_train, X_test, y_test)

GaussianNB
Misclassified samples: 92
Accuracy: 0.65
Recall score :  0.6515151515151515
Precision score :  0.6515151515151515
F1 score :  0.6515151515151515
              precision    recall  f1-score   support

           0       0.65      0.75      0.70       142
           1       0.65      0.53      0.59       122

    accuracy                           0.65       264
   macro avg       0.65      0.64      0.64       264
weighted avg       0.65      0.65      0.65       264



In [330]:
from sklearn.naive_bayes import GaussianNB 
model = GaussianNB()

kfold = KFold(n_splits = 10, random_state = 10)
scores = cross_val_score(model,X_train,y_train,cv=kfold,scoring='recall_macro')
print(scores)

print("Accuracy: %0.2f%% (+/- %0.2f%%)" % (100*scores.mean(), 100*scores.std()))

[0.69186828 0.63529412 0.78594771 0.68237179 0.66356383 0.68636364
 0.64935065 0.71572581 0.65862291 0.75494071]
Accuracy: 69.24% (+/- 4.52%)


### 1.4. Only use sig cont vars 

In [331]:
import scipy

ttest_df = pd.DataFrame(columns = ['Feature','FeatureName','t-stats','p-value','Skew','Kurtosis'])
ttest_df['Feature'] = sig_cont_vars
ttest_df['FeatureName'] = sig_cont_vars

df0 = all_df[all_df['Class'] == 0]
df1 = all_df[all_df['Class'] == 1]

for var in cont_vars:
    result = scipy.stats.ttest_ind(df0[df0[var].isna()==False][var], df1[df1[var].isna()==False][var])
    ttest_df.loc[ttest_df['Feature'] == var,'t-stats'] = result[0]
    ttest_df.loc[ttest_df['Feature'] == var,'p-value'] = result[1]
    ttest_df.loc[ttest_df['Feature'] == var,'Skew'] = all_df[var].skew()
    ttest_df.loc[ttest_df['Feature'] == var,'Kurtosis'] = all_df[var].kurt()
  
ttest_df['abs_tstats'] = np.abs(ttest_df['t-stats'])
ttest_df = ttest_df.sort_values(['abs_tstats'], ascending = False)
ttest_df = ttest_df.merge(miss_df, left_on = 'Feature',right_on='Feature',how='inner')

ttest_df

,Feature,FeatureName,t-stats,p-value,Skew,Kurtosis,abs_tstats,Count,Percentage
0,RIDAGEYR,RIDAGEYR,-10.6959,2.03864e-25,0.000859062,-1.18415,10.6959,0,0.000000
1,LBDSALSI,LBDSALSI,7.86981,8.78556e-15,-0.347124,1.00179,7.86981,119,10.144928
2,LBXSAL,LBXSAL,7.86981,8.78556e-15,-0.347124,1.00179,7.86981,119,10.144928
3,LBDSGLSI,LBDSGLSI,-7.57897,7.62166e-14,1.68589,2.68658,7.57897,119,10.144928
4,LBXSGL,LBXSGL,-7.57309,7.95584e-14,1.69302,2.7168,7.57309,119,10.144928
5,BMXWAIST,BMXWAIST,-7.55143,9.31728e-14,0.624863,0.646716,7.55143,88,7.502131
6,LBXSUA,LBXSUA,-7.52802,1.10476e-13,0.762941,0.732965,7.52802,119,10.144928
7,LBDSUASI,LBDSUASI,-7.5279,1.10568e-13,0.762869,0.733033,7.5279,119,10.144928
8,LBXLYPCT,LBXLYPCT,6.69582,3.48404e-11,0.303575,0.293664,6.69582,97,8.269395
9,BMXBMI,BMXBMI,-6.58331,7.24781e-11,1.13585,1.8663,6.58331,45,3.836317


In [332]:
sig_cont_vars = list(ttest_df[ttest_df['p-value'] <= 0.05].sort_values(['abs_tstats'],ascending=False).Feature)
print(sig_cont_vars)
print(len(sig_cont_vars))

['RIDAGEYR', 'LBDSALSI', 'LBXSAL', 'LBDSGLSI', 'LBXSGL', 'BMXWAIST', 'LBXSUA', 'LBDSUASI', 'LBXLYPCT', 'BMXBMI', 'URXUMA', 'URXUMS', 'BMXARMC', 'LBXRDW', 'LBDHDDSI', 'LBDHDD', 'INDFMPIR', 'BMXWT', 'LBXNEPCT', 'BMXLEG', 'LBDSTRSI', 'LBXSTR', 'BMXARML', 'LBDSCRSI', 'LBXSCR', 'LBXSAPSI', 'BPXSY3', 'LBXSOSSI', 'LBXTHG', 'LBDTHGSI', 'LBDNENO', 'LBXSBU', 'LBDSBUSI', 'LBXSIR', 'BPXDI2', 'LBDSIRSI', 'LBXSCH', 'LBDSCHSI', 'LBXSLDSI', 'BPXSY2', 'LBDTCSI', 'LBXTC', 'BPXDI1', 'BPXDI3', 'BMXHT', 'BPXSY1', 'LBXSGTSI', 'URXVOL1', 'LBDLYMNO', 'LBXHGB', 'BPXPLS', 'LBXWBCSI', 'LBXSTP', 'LBDSTPSI', 'LBXSGB', 'LBDSGBSI', 'LBXMOPCT', 'LBXMC', 'LBXRBCSI', 'LBXEOPCT']
60


In [333]:
len(sig_cont_vars)

60

In [334]:
X_train, X_test, y_train, y_test = train_test_split(all_df[sig_cont_vars], all_df['Class'], test_size=0.25, random_state=1)
    
NaiveBayesPrediction(X_train, y_train, X_test, y_test)

GaussianNB
Misclassified samples: 86
Accuracy: 0.67
Recall score :  0.6742424242424242
Precision score :  0.6742424242424242
F1 score :  0.6742424242424242
              precision    recall  f1-score   support

           0       0.67      0.77      0.72       142
           1       0.68      0.56      0.61       122

    accuracy                           0.67       264
   macro avg       0.68      0.67      0.67       264
weighted avg       0.68      0.67      0.67       264



In [335]:
from sklearn.naive_bayes import GaussianNB 
model = GaussianNB()

kfold = KFold(n_splits = 10, random_state = 10)
scores = cross_val_score(model,X_train,y_train,cv=kfold,scoring='recall_macro')
print(scores)

print("Accuracy: %0.2f%% (+/- %0.2f%%)" % (100*scores.mean(), 100*scores.std()))

[0.71370968 0.65359477 0.7748366  0.61891026 0.65857713 0.675
 0.66071429 0.73185484 0.66023166 0.76581028]
Accuracy: 69.13% (+/- 4.96%)


### 1.5. Remove features with high correlation

In [336]:
# build a dataframe with the correlation between features
# remember that the absolute value of the correlation
# coefficient is important and not the sign

corrmat = all_df[sig_cont_vars].corr()
corrmat = corrmat.abs().unstack() # absolute value of corr coef
corrmat = corrmat.sort_values(ascending=False)

corrmat = pd.DataFrame(corrmat).reset_index()
corrmat.columns = ['feature1', 'feature2', 'corr']
corrmat['MissingF1'] = corrmat.feature1.apply(lambda x:MissingPercentage(x))
corrmat['MissingF2'] = corrmat.feature2.apply(lambda x:MissingPercentage(x))
corrmat.head()

,feature1,feature2,corr,MissingF1,MissingF2
0,LBXSTP,LBDSTPSI,1.0,0.101449,0.101449
1,LBDSTPSI,LBXSTP,1.0,0.101449,0.101449
2,LBDSALSI,LBXSAL,1.0,0.101449,0.101449
3,LBXSAL,LBDSALSI,1.0,0.101449,0.101449
4,LBDSGBSI,LBXSGB,1.0,0.101449,0.101449


In [337]:
correlated_groups = corrmat[corrmat['corr'] > 0.90]
correlated_groups

,feature1,feature2,corr,MissingF1,MissingF2
0,LBXSTP,LBDSTPSI,1.000000,0.101449,0.101449
1,LBDSTPSI,LBXSTP,1.000000,0.101449,0.101449
2,LBDSALSI,LBXSAL,1.000000,0.101449,0.101449
3,LBXSAL,LBDSALSI,1.000000,0.101449,0.101449
4,LBDSGBSI,LBXSGB,1.000000,0.101449,0.101449
5,LBXSGB,LBDSGBSI,1.000000,0.101449,0.101449
6,LBXSCH,LBXSCH,1.000000,0.101449,0.101449
7,LBXTC,LBXTC,1.000000,0.098039,0.098039
8,LBDTCSI,LBDTCSI,1.000000,0.098039,0.098039
9,BPXSY2,BPXSY2,1.000000,0.101449,0.101449


In [338]:
sig_cont_vars = []
remaining_vars = correlated_groups.feature1.unique()

while(len(remaining_vars) > 0):
    feature = remaining_vars[0]
    correlated_block = correlated_groups[correlated_groups.feature1 == feature]
    min_ind = correlated_block[['MissingF2']].idxmin() 
    sel_var = correlated_block.feature2[min_ind].values[0]
    removed_vars = [var for var in list(correlated_block.feature2.values)]
    remaining_vars = [var for var in remaining_vars if var not in removed_vars]
    if sel_var not in sig_cont_vars:
        sig_cont_vars = sig_cont_vars + [sel_var]    
    
print(sig_cont_vars)
len(sig_cont_vars)

['LBDSTPSI', 'LBXSAL', 'LBXSGB', 'LBDTCSI', 'BPXSY2', 'LBXSLDSI', 'LBXEOPCT', 'LBDSIRSI', 'BPXDI2', 'BPXDI3', 'LBDSBUSI', 'BPXDI1', 'LBXSGTSI', 'BMXHT', 'BPXSY1', 'LBDTHGSI', 'URXVOL1', 'LBDLYMNO', 'LBXHGB', 'BPXPLS', 'LBXWBCSI', 'LBXMC', 'BMXARMC', 'LBDSGLSI', 'BMXWAIST', 'LBXSUA', 'LBXLYPCT', 'BMXWT', 'URXUMA', 'LBXRDW', 'LBXSOSSI', 'LBDHDDSI', 'INDFMPIR', 'BMXLEG', 'LBDSTRSI', 'BMXARML', 'LBDSCRSI', 'LBXSAPSI', 'LBXMOPCT', 'RIDAGEYR', 'LBXRBCSI']


41

In [339]:
X_train, X_test, y_train, y_test = train_test_split(all_df[sig_cont_vars], all_df['Class'], test_size=0.25, random_state=1)
    
NaiveBayesPrediction(X_train, y_train, X_test, y_test)

GaussianNB
Misclassified samples: 85
Accuracy: 0.68
Recall score :  0.678030303030303
Precision score :  0.678030303030303
F1 score :  0.678030303030303
              precision    recall  f1-score   support

           0       0.67      0.80      0.73       142
           1       0.69      0.54      0.61       122

    accuracy                           0.68       264
   macro avg       0.68      0.67      0.67       264
weighted avg       0.68      0.68      0.67       264



In [340]:
from sklearn.naive_bayes import GaussianNB 
model = GaussianNB()

kfold = KFold(n_splits = 10, random_state = 10)
scores = cross_val_score(model,X_train,y_train,cv=kfold,scoring='recall_macro')
print(scores)

print("Accuracy: %0.2f%% (+/- %0.2f%%)" % (100*scores.mean(), 100*scores.std()))

[0.70799731 0.64248366 0.74901961 0.64391026 0.64793883 0.72922078
 0.66363636 0.69489247 0.64350064 0.7397892 ]
Accuracy: 68.62% (+/- 4.09%)


In [341]:
from mixed_naive_bayes import MixedNB

X_train, X_test, y_train, y_test = train_test_split(all_df[sig_cont_vars], all_df['Class'], test_size=0.25, random_state=1)

def MixedBayesPrediction(X_train, y_train, X_test, y_test):
    clf = MixedNB()
    clf.fit(X_train, y_train)
    
    y_pred_gnb = clf.predict(X_test)
    y_prob_pred_gnb = clf.predict_proba(X_test)
    # how did our model perform?
    count_misclassified = (y_test != y_pred_gnb).sum()
    
    print("GaussianNB")
    print("=" * 30)
    print('Misclassified samples: {}'.format(count_misclassified))
    accuracy = accuracy_score(y_test, y_pred_gnb)
    print('Accuracy: {:.2f}'.format(accuracy))
    
    print("Recall score : ", recall_score(y_test, y_pred_gnb , average='micro'))
    print("Precision score : ",precision_score(y_test, y_pred_gnb , average='micro'))
    print("F1 score : ",f1_score(y_test, y_pred_gnb , average='micro'))
    
    print(classification_report(y_test, y_pred_gnb))
    
MixedBayesPrediction(X_train, y_train, X_test, y_test)

GaussianNB
Misclassified samples: 85
Accuracy: 0.68
Recall score :  0.678030303030303
Precision score :  0.678030303030303
F1 score :  0.678030303030303
              precision    recall  f1-score   support

           0       0.67      0.80      0.73       142
           1       0.69      0.54      0.61       122

    accuracy                           0.68       264
   macro avg       0.68      0.67      0.67       264
weighted avg       0.68      0.68      0.67       264

